In [4]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

## Basic Game

Das Modul `Chess.jl` stellt eine Implementation des Schachspiels für die Sprache Julia bereit. Dieses kann gegebenenfalls über den Packagemanager `Pkg` installiert werden.

In [ ]:
using Pkg
Pkg.add("Chess")

In [5]:
using Chess

Ein Schachspiel besteht aus einem abwechselnden Durchführen jeweils genau eines Zuges. Besondere Konventionen wie zum Beispiel Rochaden zählen hierbei als ein Zug.

Hierbei folgt das Spielen eines Schachspiels stets der folgenden Logik:

ai_turn = false

while !isterminal(game)

    move = next_move(game, ai_turn ? ai : player)
    domove!(game, move)
    ai_turn = !ai_turn
    
end     

Die Funktion `next_move` : (`board`, `entity`) -> `move` bildet auf den nächsten durchzuführenden Zug ab. Hierbei wird die Implementation der Funktion über Multiple-Dispatch mit dem `entity` Attribut bestimmt. Dieses welchselt mit der Partei am Zug.

### Entity Player

Die Spieler-Implementation (Methode) der `next_move` Funktion setzt einer Benutzer-Eingabe voraus, welche in Julia mithilfe der `readline`(`stream`) Funktion erreicht werden kann. 

In [ ]:
input = readline(stdin) # remove -> move to UsingJulia
print(input)

Wie bereits zu erkennen ist, kann der Benutzer jeden beliebigen Text als Eingabe vormulieren, auch solche, die nicht einer validen Zugeingabe entsprechen. Um einer solchen fehlerhaften Eingabe entgegen zu wirken, definieren wir die folgende Funktion:

In [6]:
function validate_userinput(input::String)
    # check pawn -> f.e. queen
    if match(r"^[a-hA-H][1-8][a-hA-H][1-8]$", input) === nothing
        return "Invalid input '$(input)'"
    end
    return nothing
end

validate_userinput (generic function with 1 method)

Der reguläre Ausdruck `^[a-h][1-8][a-h][1-8]$` akzeptiert hierbei nur Eingabetexte, welche aus zwei Feldern bestehen: Start & Ende. Da es sich bei der X- und Y-Indizierung auf einem Schachbrett um Koordinaten im Bereich [a-h] und [a-8] handelt, werden diese zusätzlich auf die entsprechenden Bereiche begrenzt. Je nach Status des Spiels ist eine Angabe des Startfelden theoretisch redundant, jedoch entschieden wir uns, die verbose Form als Eingabenorm beizubehalten.

Falls die Eingabe der `validate_userinput` Funktion einen syntaktisch validen Zug aufweist, so gibt diese `nothing` zurück. Der Rückgabewert ist als eine Art Felher zu interpretieren. Ist die Eingabe invalid, so wird ein String mit der Fehlernachricht zurückgegeben.

Des weiteren ist zu prüfen, ob es sich bei dem syntaktisch validen Zug um einen tatsächlich legalen und durchführbaren Zug handelt. Beispielsweise könnte der Benutzer in der Startposition den Zug `"e2f2"` eingeben, welcher dem Bauer instruktieren würde, um ein Feld nach rechts zu gehen. Dieser Zug ist laut Spielregeln nicht möglich und muss daher unterbunden werden. Hierzu definieren wir die folgende Funktion:

In [8]:
function validate_legal_move(board::Board, move::Move) # join with other function
    all_legal_moves = moves(board)
    if move ∉ all_legal_moves
        return("Illegal move '$(move)'")
    end
    return nothing
end

validate_legal_move (generic function with 1 method)

Mithilfe der `moves(board)` Funktion können wir alle legalen Züge auflisten. Anschließend ist zu prüfen, ob sich der Zugkandidat in der entsprechenden Liste befindet. In Julia kann dies durch die respektiven Operatoren `∈` und `∉` durchgeführt werden. Die `validate_legal_move` Funktion gibt bei einem Fehler ebenfalls einen String zurück, andersweitig `nothing`.

Mit den beiden Prüffunktionen können wir anschließend die `next_move` Funktion für einen Player definieren:

In [9]:
struct Player end # explain why

function next_move(board, _::Player)
    move = nothing
    while true
        userinput = readline(stdin)
        if userinput == "exit" || userinput == "reset"
            return userinput
        end
        error = validate_userinput(userinput)
        if error != nothing
            println(error)
            continue
        end
        move = movefromstring(userinput)
        error = validate_legal_move(board, move)
        if error != nothing
            println(error)
            continue
        end
        break
    end
    return move
end

next_move (generic function with 1 method)

Als zusätzlich valide Eingaben definieren wir die Worte `exit` und `reset`, welche gegebenenfalls das Spiel vorzeitig beenden oder zurücksetzten können.

## Game Implementation

Mit allen bereits implementieren Komponenten lässt sich eine erste, lauffähige Durchführung eines Schachspiels implementieren:

In [10]:
function play_game(ai, player = Player(), game = SimpleGame(), ai_turn = false)
    print_game(game)
    
    while !isterminal(game)
        move = next_move(board(game), ai_turn ? ai : player)
        if move == "exit"
            IJulia.clear_output()
            return board(game)
        end
        if move == "reset"
            return play_game(ai)
        end
        domove!(game, move)
        print_game(game)
        ai_turn = !ai_turn
    end
    
    return isdraw(game) ? "Draw!" : ai_turn ? "Checkmate! You win!" : "Checkmate! You lost!"
end

play_game (generic function with 4 methods)

Nachdem ein Spiel beendet wurde, gibt die Funktion das Ergebnis des Spiels zurück.

Um die Ausgabe während des Spiels zu vereinfachen, definieren wir die folgende Funktion, welche die aktuelle Ausgabezelle löscht und das Spielbrett erneut ausgibt. Konvertiert man ein `Chess.jl`- Schachbrett zu HTML, so wird dieses in einer anschaulichen Form ausgerendert.

In [12]:
function print_game(game)
    IJulia.clear_output()
    html = HTML(IJulia.html(board(game)))
    IJulia.display(html)
end

print_game (generic function with 1 method)

Als letzte Komponente für ein vollständiges Schachspiel fehlt ein KI-Gegner. Hierzu importieren wir die `RandomAI`.

In [11]:
using Pkg
Pkg.add("NBInclude")

    Updating registry at `C:\Users\flori\.julia\registries\General.toml`
   Resolving package versions...
   Installed NBInclude ─ v2.3.0
    Updating `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  [0db19996] + NBInclude v2.3.0
    Updating `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
  [0db19996] + NBInclude v2.3.0
Precompiling project...
  ✓ NBInclude
  1 dependency successfully precompiled in 2 seconds. 51 already precompiled.


In [13]:
using NBInclude
@nbinclude("MiniMaxAI.ipynb")

   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flori\.julia\environments\v1.8\Manifest.toml`


next_move (generic function with 2 methods)

In [14]:
play_game(MiniMaxAI())

HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"2\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"3\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"2\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"3\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bq.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\

"Checkmate! You win!"

***

by Florian Stach and Luc Kaiser

Last updated: 30/11/2022

***